# Build a Crew to Tailor Job Applications

In this lesson, you will built your first multi-agent system.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [4]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [98]:
import os
# from utils import get_openai_api_key, get_serper_api_key

openai_api_key = "sk-proj-zLEsC4gv5zJ1EWsn3pgUT3BlbkFJrIDdyUFGu2GPzwjhVfbv"
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["SERPER_API_KEY"] = "52a7b98fc031f9e0a3b5aba2ef667a9c5db81d17"

## crewAI Tools

In [125]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  DirectoryReadTool,
  DirectorySearchTool,
  PDFSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = DirectoryReadTool(file_path='./data')
search_resume = DirectorySearchTool(file_path='./data')
semantic_search_resume = PDFSearchTool()

- Uncomment and run the cell below if you wish to view `fake_resume.md` in the notebook.

In [100]:
# from IPython.display import Markdown, display
# display(Markdown("./fake_resume.md"))

## Creating Agents

In [126]:
# Agent 1: Researcher
researcher = Agent(
    role="Tech Job Researcher",
    goal="Make sure to do amazing analysis on "
         "job posting to help job applicants",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Job Researcher, your prowess in "
        "navigating and extracting critical "
        "information from job postings is unmatched."
        "Your skills help pinpoint the necessary "
        "qualifications and skills sought "
        "by employers, forming the foundation for "
        "effective application and cover letter tailoring."
    )
)

In [127]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do incredible research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume enhancements."
    )
)

In [128]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Y-Combinator Resume Strategist for ML/AI Engineers",
    goal="Find all the best ways to make a "
         "resume stand out in the job market.",
    tools = [scrape_tool, search_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at refining resumes to highlight the most "
        "relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
        "You understand what is important to startup founders"
        "(especially those in y-combinator) in the ML/AI space,"
        "and you know how to make a resume which will get noticed by them."
    )
)

In [129]:
# Agent 4: Cover Letter Strategist
cover_letter_composer = Agent(
    role="Y-Combinator Cover Letter Composer for ML/AI Engineers",
    goal="Compose a cover letter for a Y-Combinator startup application",
    tools = [scrape_tool,
             read_resume, search_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "With a strategic mind and an eye for detail, you "
        "excel at composing cover letters for y-combinator startups."
        "You understand what is important to startup founders"
        "(especially those in y-combinator) in the ML/AI space."
        "You know how to highlight relevant skills and experiences, ensuring they "
        "resonate perfectly with the job's requirements."
    )
)

## Creating Tasks

In [154]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract key skills, experiences, and qualifications "
        "required. Use the tools to gather content and identify "
        "and categorize the requirements."
    ),
    expected_output=(
        "A structured list of job requirements, including necessary "
        "skills, qualifications, and experiences, as well as preferred "
        "skills qualifications and experiences."
    ),
    agent=researcher,
    async_execution=True
)

In [163]:
# TODO: cut out the middle man with this agent and just skip to the cover letter/resume strategists

# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed and comprehensive personal and professional profile "
        # "using the LinkedIn ({linkedin_url}) profile"
        "using only the and resume / portfolio"
        "files located in ({portfolio_dir}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler,
    async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [164]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, tailor the resume to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "resume content. Make sure this is the best resume ever but "
        "don't make up any information. Update every section, "
        "inlcuding the initial summary, work experience, skills, "
        "and education. All to better reflect the candidate's "
        "abilities and how it matches the job posting."
    ),
    expected_output=(
        "An updated resume that effectively highlights the candidate's "
        "qualifications and experiences relevant to the job."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [165]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
cover_letter_compose_task = Task(
    description=(
        "Using the profile and job requirements obtained from "
        "previous tasks, write the cover letter to highlight the most "
        "relevant areas. Employ tools to adjust and enhance the "
        "cover letter content. Make sure the cover letter is written"
        "specifically for the role described in the job posting ({job_posting_url})"
        "and includes only truthful information (i.e. do not make up information)."
    ),
    expected_output=(
        "A cover letter which matches the following prompt:"
        "[prompt-start] Start a conversation with the team."
        "Share something about you, what you're looking for,"
        "or why the company interests you. [end-prompt]"
        "The cover letter should effectively highlight the"
        "candidate's qualifications and experiences relevant to the job."
        "Be sure to keep the cover letter concise (less than 130 words)"
        "and to the point, highlighting only those skills and experiences"
        "which are most relevant to the job posting ({job_posting_url})."
        "Tailor the cover letter to the specific role and company."
        "Express why you are interested in the role and how your"
        "experience aligns with the job requirements."
    ),
    output_file="cover_letter.md",
    context=[research_task, profile_task],
    agent=cover_letter_composer
)


In [166]:
# Todo: add a reviewer to review for factual information

## Creating the Crew

In [167]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

In [168]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            # resume_strategist,
            cover_letter_composer
            ],

    tasks=[research_task,
           profile_task,
        #    resume_strategy_task,
           cover_letter_compose_task
           ],
    manager_llm=ChatOpenAI(model="gpt-4-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

2024-05-24 14:38:50,289 - 8671874048 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

- Set the inputs for the execution of the crew.

In [169]:
job_application_inputs = {
    'job_posting_url': 'https://www.workatastartup.com/jobs/66368',
    # 'github_url': 'https://github.com/sethcoast',
    # 'linkedin_url': 'https://www.linkedin.com/in/seth-donaldson/',
    'portfolio_dir': './data',
    # 'personal_writeup': """Seth is an accomplished Machine Learning Engineer
    # with 2 years of experience in ML infrastructure engineering; 4 years 
    # of internship and research experience in deep learning for computer vision;
    # and 2 years of experience as a mobile app developer. He is proficient in
    # python, scala, and java, and has experience with a variety of machine
    # learning libraries and frameworks (including both PyTorch and TensorFlow).
    # Seth holds a Master's degree in Computer Science from Columbia University.
    # He is passionate about learning languages. He has learned Brazilian Portuguese,
    # and is currently learning Japanese. Seth is a quick learner and a team player. He is
    # looking for opportunities to apply his skills in a dynamic and innovative
    # startup environment."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [170]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Analyze the job posting URL provided (https://www.workatastartup.com/jobs/66368) to extract key skills, experiences, and qualifications required. Use the tools to gather content and identify and categorize the requirements.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Compile a detailed and comprehensive personal and professional profile using only the and resume / portfoliofiles located in (./data). Utilize tools to extract and synthesize information from these sources.
 [DEBUG]: [Crew Manager] Task output: None
 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Using the profile and job requirements obtained from previous tasks, write the cover letter to highlight the most relevant areas. Employ tools to adjust and enhance the cover letter content. Make sure the cover letter is writtenspecifically for the role described in the job posting (https://www.w

- Dislplay the generated `tailored_resume.md` file.

In [138]:
from IPython.display import Markdown, display
display(Markdown("./cover_letter.md"))

Dear Dream3D Team,

I am writing to express my interest in the ML Engineer position at Dream3D. With 3+ years of experience in machine learning engineering, a track record of implementing research papers in PyTorch, and a solid understanding of generative models such as VAEs and GANs, I believe I am well-equipped to contribute to your team. Your innovative approach to building generative models for virtual worlds aligns perfectly with my passion for exploring intelligent systems. I am excited about the opportunity to work with a creative team in Brooklyn, and I am flexible with hybrid work schedules. I look forward to the possibility of joining your founding team and contributing to the future of digital worlds.

Sincerely,
Seth Donaldson

- Dislplay the generated `interview_materials.md` file.

In [ ]:
display(Markdown("./interview_materials.md"))